# Preprocessing #9

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn import set_config
set_config(transform_output='pandas')

from category_encoders import OrdinalEncoder

from src.helpers import *

In [2]:
# import data

df = pd.read_csv('../data/train.csv')

In [3]:
# check data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# ---Data Cleaning---

In [4]:
# Converting integers to string for the OrdinalEncoder

def convert_int_to_string(data, columns):
  for column in columns:
    data[column] = data[column].astype(str)
  return data

In [5]:
# Define features to convert and convert

columns_to_convert = ['OverallQual', 'OverallCond']

df = convert_int_to_string(df, columns_to_convert)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   object 
 18  OverallC

In [6]:
# OutlierRemover is a custom transformer imported from src/helpers.py
# It can remove outliers from multiple columns by specifying the iqr_multipler

outlier_cols = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','TotalBsmtSF','1stFlrSF','GrLivArea',
                'OpenPorchSF']

outlier_remover = OutlierRemover(columns=outlier_cols,iqr_multiplier=10)

outlier_remover.fit(df)
df_proc = outlier_remover.transform(df)

df_proc.count()

Id               1449
MSSubClass       1449
MSZoning         1449
LotFrontage      1196
LotArea          1449
                 ... 
MoSold           1449
YrSold           1449
SaleType         1449
SaleCondition    1449
SalePrice        1449
Length: 81, dtype: int64

# ---Pipeline---

In [7]:
# try dropping all OE & OHE features
# define the OE & OHE columns and add them to the columns to drop filter

oe_cols = ['OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
           'BsmtFinType1','BsmtFinType2','HeatingQC','FireplaceQu', 'GarageFinish','GarageQual', 'GarageCond',
           'PoolQC','KitchenQual','Fence']

ohe_cols = ['MSSubClass','MSZoning','Alley','LotShape','LandContour','LotConfig','LandSlope',
            'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
            'Exterior2nd','MasVnrType','Foundation','Heating', 'CentralAir','Electrical','Functional',
            'GarageType','PavedDrive','MiscFeature','MoSold','YrSold','SaleType','SaleCondition']

In [8]:
# Defining the list of columns that will go into the pipeline
# Remove columns that should not be included in the pipeline

all_columns = df.columns.tolist()
columns_remove_from_filter = ['SalePrice','Id','Street','Utilities','LowQualFinSF','GarageYrBlt',
                              '3SsnPorch']

# Add the OE columns to the remove filter
columns_remove_from_filter.extend(oe_cols)
columns_remove_from_filter.extend(ohe_cols)

columns_to_keep = [item for item in all_columns if item not in columns_remove_from_filter]

In [9]:
# Preprocessor to filter unwanted columns, or unexpected columns.

preprocessor_filter = ColumnTransformer([
  ('passthrough','passthrough',columns_to_keep)
],remainder='drop', verbose_feature_names_out=False)

In [10]:
# Preprocessor for imputing all null variables

# separating the OHE and OE columns to remove OE features from the preprocessor
cols_impute_na_ohe = ['Alley', 'GarageType', 'MiscFeature']
cols_impute_na_oe = ['Fence','BsmtQual', 'BsmtCond', 'BsmtExposure','BsmtFinType1','BsmtFinType2', 'FireplaceQu',
                     'GarageFinish', 'GarageQual','GarageCond','PoolQC']

cols_impute_na = cols_impute_na_ohe
cols_impute_none = ['MasVnrType']
cols_impute_most_freq = ['Electrical']
cols_impute_zero = ['LotFrontage', 'MasVnrArea']

preprocessor_imputer = ColumnTransformer([
  # disable all imputer related to OE and OHE

  # ('impute_na', SimpleImputer(strategy='constant', fill_value='NA'), cols_impute_na),
  # ('impute_none', SimpleImputer(strategy='constant', fill_value='None'), cols_impute_none),
  ('impute_zero', SimpleImputer(strategy='constant', fill_value=0), cols_impute_zero),
  # ('impute_most_freq', SimpleImputer(strategy='most_frequent'), cols_impute_most_freq)

], remainder='passthrough', verbose_feature_names_out=False)

In [11]:
# Defining maps for Ordinal categories

dict_na_ex_6 = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
dict_na_gd_5 = {'NA':0,'No':1,'Mn':2,'Av':3,'Gd':4}
dict_bsmt = {'NA':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}
dict_garage = {'NA':0,'Unf':1,'RFn':2,'Fin':3}
dict_1_10 = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6, 8:7, 9:8, 10:9}
dict_fence = {'NA':0, 'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4}


ordinal_cat_map = [
  {'col':'BsmtQual','mapping':dict_na_ex_6},
  {'col':'BsmtCond','mapping':dict_na_ex_6},
  {'col':'BsmtExposure','mapping':dict_na_gd_5},
  {'col':'BsmtFinType1','mapping':dict_bsmt},
  {'col':'BsmtFinType2','mapping':dict_bsmt},
  {'col':'FireplaceQu','mapping':dict_na_ex_6},
  {'col':'GarageFinish','mapping':dict_garage},
  {'col':'GarageQual','mapping':dict_na_ex_6},
  {'col':'GarageCond','mapping':dict_na_ex_6},
  {'col':'PoolQC','mapping':dict_na_ex_6},
  {'col': 'OverallQual', 'mapping':dict_1_10},
  {'col': 'OverallCond', 'mapping':dict_1_10},
  {'col': 'ExterQual', 'mapping': dict_na_ex_6},
  {'col': 'ExterCond', 'mapping': dict_na_ex_6},
  {'col': 'HeatingQC', 'mapping': dict_na_ex_6},
  {'col': 'KitchenQual', 'mapping': dict_na_ex_6},
  {'col': 'Fence', 'mapping': dict_fence}
  ]

In [12]:
# Preprocessor for encoding and scaling

ohe_cols = ['MSSubClass','MSZoning','Alley','LotShape','LandContour','LotConfig','LandSlope',
            'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
            'Exterior2nd','MasVnrType','Foundation','Heating', 'CentralAir','Electrical','Functional',
            'GarageType','PavedDrive','MiscFeature','MoSold','YrSold','SaleType','SaleCondition']

oe_cols = ['OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
           'BsmtFinType1','BsmtFinType2','HeatingQC','FireplaceQu', 'GarageFinish','GarageQual', 'GarageCond',
           'PoolQC','KitchenQual','Fence']

robust_cols = ['LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',
               'GrLivArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','ScreenPorch','PoolArea','MiscVal']

preprocessor_encode_scale = ColumnTransformer([
  #disabling the OHE to drop all features using OHE
  # ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ohe_cols),

  #disabling the OE to drop all features using OE
  # ('oe', OrdinalEncoder(), oe_cols),

  ('standard_scaler', StandardScaler(), ['LotFrontage','GarageArea']),
  ('robust_scaler', RobustScaler(), robust_cols),
  ('minmax_scaler', MinMaxScaler(), ['YearBuilt','YearRemodAdd'])
], remainder='passthrough', verbose_feature_names_out=False)

In [13]:
# split train and val

X = df_proc.drop(columns='SalePrice')
y = df_proc['SalePrice'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# run the main pipeline with all preprocessors

main_pipe = Pipeline([
  ('preprocessor_filter', preprocessor_filter),
  ('preprocessor_imputer', preprocessor_imputer),
  ('preprocessor_encode_scale', preprocessor_encode_scale)
])

In [15]:
# fit and transform X_train and X_val

main_pipe.fit(X_train)

X_train_proc = main_pipe.transform(X_train)
X_val_proc = main_pipe.transform(X_val)

In [16]:
# checking the number of columns in X_train after processing

len(X_train_proc.columns)

28

### XG Boost Regressor

In [21]:
# Import, fit on model, then save results

from xgboost import XGBRegressor

xgb = XGBRegressor()
xgb.fit(X_train_proc, y_train)

linear_results_train = evaluate_regression(xgb, X_train_proc, y_train, 'xgb')
linear_results_val = evaluate_regression(xgb, X_val_proc, y_val, 'xgb')

In [22]:
# result of train

linear_results_train

,MAE,MSE,RMSE,MAPE,R2,adj_r2
xgb,918.202454,1912030.0,1382.761719,0.00611,0.999693,0.999686


In [23]:
# result of val

linear_results_val

,MAE,MSE,RMSE,MAPE,R2,adj_r2
xgb,18722.386719,779775936.0,27924.46875,0.118908,0.872117,0.858398


In [25]:
# calculating the log RMSE

val_preds = xgb.predict(X_val_proc)
pred_log = np.log(val_preds)
y_log = np.log(y_val)
root_mean_squared_error(y_log, pred_log)

0.16392774242225222